# DEVELOPING A TRADING STRATEGY FOR THE LARGEST STOCKS BY MARKET CAPITALIZATION IN THE COLOMBO STOCK EXCHANGE USING UNSUPERVISED LEARNING

In [ ]:
# description

# Importing the necessary libraries

In [4]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import yfinance as yf

# Data preparation

## Read Company data obtained from simplywall.com

In [9]:
cse = pd.read_csv('cse_companies.csv')
cse.head()

,Ticker Symbol,Company Name,Last Price,7D Return,1Y Return,Market Cap,Analysts Target,Valuation,Growth,Div Yield,Industry
0,EXPO.N0000,Expolanka Holdings,LK₨150.50,0.000,0.132,LK₨294.2b,NaN,PS1.2x,NaN,2.30%,Transportation
1,JKH.N0000,John Keells Holdings,LK₨193.25,-0.004,0.157,LK₨289.6b,LK₨252.97,PE25.8x,E55.5%,0.80%,Capital Goods
2,CTC.N0000,Ceylon Tobacco,"LK₨1,235.00",0.004,0.345,LK₨231.3b,"LK₨1,256.73",PE8.1x,E7.9%,12.00%,"Food, Beverage & Tobacco"
3,LOLC.N0000,LOLC Holdings,LK₨439.00,-0.005,-0.11,LK₨208.6b,NaN,PE7x,NaN,0%,Diversified Financials
4,LOFC.N0000,LOLC Finance,LK₨6.10,0.000,0.017,LK₨201.8b,NaN,PE9.4x,NaN,NaN,Diversified Financials


## List of Industries

In [11]:
cse.Industry.unique()

array(['Transportation', 'Capital Goods', 'Food, Beverage & Tobacco',
       'Diversified Financials', 'Telecom', 'Banks', 'Consumer Retailing',
       'Energy', 'Insurance', 'Consumer Durables', 'Materials',
       'Healthcare', 'Utilities', 'Consumer Services',
       'Real Estate Management and Development', 'Retail', 'Software',
       'Automobiles', 'Household', 'Commercial Services'], dtype=object)

## List of Ticker Symbols

In [12]:
cse['Symbol'] = cse['Ticker Symbol'].str.replace('.' , '-') + '.CM'
tickers_list = cse['Symbol'].tolist()
tickers_list[0:5]

['EXPO-N0000.CM',
 'JKH-N0000.CM',
 'CTC-N0000.CM',
 'LOLC-N0000.CM',
 'LOFC-N0000.CM']

## Get the stock data of 10 years from Yahoo Finance

In [39]:
start_date = '2014-01-01'
end_date = '2023-12-31'

df = yf.download(tickers=tickers_list,
                 start=start_date,
                 end=end_date).stack(future_stack=True)
df.head()

[*********************100%%**********************]  283 of 283 completed

14 Failed downloads:
['CBNK-N0000.CM']: YFChartError("%ticker%: Data doesn't exist for startDate = 1388514600, endDate = 1703961000")
['CALT-N0000.CM', 'HBS-N0000.CM', 'LUMX-N0000.CM', 'LCBF-N0000.CM', 'EXT-N0000.CM', 'SWAD-N0000.CM', 'HVA-N0000.CM', 'LGIL-N0000.CM', 'GSF-N0000.CM', 'LPL-N0000.CM', 'COOP-N0000.CM', 'FCT-N0000.CM', 'HELA-N0000.CM']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')


Price                     Adj Close       Close        High         Low  \
Date       Ticker                                                         
2014-01-02 AAF-N0000.CM   18.750000   19.607843   20.588236   18.627451   
           AAIC-N0000.CM   5.422132    8.550000    8.550000    8.540000   
           ABAN-N0000.CM  83.430313  113.900002  116.000000  108.000000   
           ABL-N0000.CM         NaN         NaN         NaN         NaN   
           ACAP-N0000.CM  17.000000   17.000000   17.100000   16.500000   

Price                           Open    Volume  
Date       Ticker                               
2014-01-02 AAF-N0000.CM    18.627451  468650.0  
           AAIC-N0000.CM    8.540000   34550.0  
           ABAN-N0000.CM  108.000000   22821.0  
           ABL-N0000.CM          NaN       NaN  
           ACAP-N0000.CM   17.100000    3000.0

In [54]:
adj_close_df = df[['Adj Close']]

# Unstack the DataFrame to create columns for each ticker
adj_close_df = adj_close_df.unstack(level='Ticker')

adj_close_df.head()

Price         adj close                                           \
Ticker     AAF-N0000.CM AAIC-N0000.CM ABAN-N0000.CM ABL-N0000.CM   
Date                                                               
2014-01-02      18.7500      5.422132     83.430313          NaN   
2014-01-03      17.8125      5.510916     80.939865          NaN   
2014-01-06      17.8125      5.510916     79.108650          NaN   
2014-01-07      17.8125      5.453841     80.573616          NaN   
2014-01-08      17.8125      5.948492     80.573616          NaN   

Price                                                                          \
Ticker     ACAP-N0000.CM ACL-N0000.CM ACME-N0000.CM AEL-N0000.CM AFS-N0000.CM   
Date                                                                            
2014-01-02          17.0    12.481286      5.246940    15.698577          NaN   
2014-01-03          17.0    12.481286      5.300480    15.489264          NaN   
2014-01-06          17.0    12.442883      5.086319    15.349717          NaN   
2014-01-07          17.9    12.673306      5.086319    15.279943          NaN   
2014-01-08          17.4    12.769316      5.246940    15.210176          NaN   

Price                     ...                                           \
Ticker     AFSL-N0000.CM  ... UDPL-N0000.CM UML-N0000.CM VFIN-N0000.CM   
Date                      ...                                            
2014-01-02     27.131100  ...     25.700001    43.574532      4.568280   
2014-01-03     27.314419  ...     26.799999    44.683014      4.630013   
2014-01-06     27.314419  ...     28.000000    40.363781      4.506546   
2014-01-07     26.947783  ...     25.799999    42.504284      4.506546   
2014-01-08     26.397827  ...     27.000000    43.192299      4.676313   

Price                                                              \
Ticker     VLL-N0000.CM VONE-N0000.CM VPEL-N0000.CM WAPO-N0000.CM   
Date                                                                
2014-01-02     1.367868     12.974626      1.657422     27.268274   
2014-01-03     1.406950     12.821079      1.629798     27.268274   
2014-01-06     1.367868     12.590760      1.602174     26.548319   
2014-01-07     1.367868     12.744308      1.602174     27.718248   
2014-01-08     1.367868     12.897853      1.629798     26.908297   

Price                                                 
Ticker     WATA-N0000.CM WIND-N0000.CM YORK-N0000.CM  
Date                                                  
2014-01-02      3.682552           NaN    132.398880  
2014-01-03      3.682552           NaN    129.581909  
2014-01-06      3.682552           NaN    128.642899  
2014-01-07      3.682552           NaN    127.703896  
2014-01-08      3.793029           NaN    130.520905  

[5 rows x 283 columns]

## Handle missing values

In [57]:
# Check for NaN in the first row of each column
columns_to_drop = adj_close_df.columns[adj_close_df.iloc[0].isna()]

# Drop the identified columns
adj_close_df = adj_close_df.drop(columns=columns_to_drop)
adj_close_df.head()

Price         adj close                                            \
Ticker     AAF-N0000.CM AAIC-N0000.CM ABAN-N0000.CM ACAP-N0000.CM   
Date                                                                
2014-01-02      18.7500      5.422132     83.430313          17.0   
2014-01-03      17.8125      5.510916     80.939865          17.0   
2014-01-06      17.8125      5.510916     79.108650          17.0   
2014-01-07      17.8125      5.453841     80.573616          17.9   
2014-01-08      17.8125      5.948492     80.573616          17.4   

Price                                                             \
Ticker     ACL-N0000.CM ACME-N0000.CM AEL-N0000.CM AFSL-N0000.CM   
Date                                                               
2014-01-02    12.481286      5.246940    15.698577     27.131100   
2014-01-03    12.481286      5.300480    15.489264     27.314419   
2014-01-06    12.442883      5.086319    15.349717     27.314419   
2014-01-07    12.673306      5.086319    15.279943     26.947783   
2014-01-08    12.769316      5.246940    15.210176     26.397827   

Price                                   ...                              \
Ticker     AGAL-N0000.CM AGST-N0000.CM  ... UCAR-N0000.CM UDPL-N0000.CM   
Date                                    ...                               
2014-01-02     21.858088      3.246157  ...    294.823090     25.700001   
2014-01-03     21.949543      3.179909  ...    294.823090     26.799999   
2014-01-06     21.034979      3.179909  ...    294.889252     28.000000   
2014-01-07     21.949543      3.179909  ...    295.484161     25.799999   
2014-01-08     21.766630      3.246157  ...    301.764008     27.000000   

Price                                                             \
Ticker     UML-N0000.CM VFIN-N0000.CM VLL-N0000.CM VONE-N0000.CM   
Date                                                               
2014-01-02    43.574532      4.568280     1.367868     12.974626   
2014-01-03    44.683014      4.630013     1.406950     12.821079   
2014-01-06    40.363781      4.506546     1.367868     12.590760   
2014-01-07    42.504284      4.506546     1.367868     12.744308   
2014-01-08    43.192299      4.676313     1.367868     12.897853   

Price                                                               
Ticker     VPEL-N0000.CM WAPO-N0000.CM WATA-N0000.CM YORK-N0000.CM  
Date                                                                
2014-01-02      1.657422     27.268274      3.682552    132.398880  
2014-01-03      1.629798     27.268274      3.682552    129.581909  
2014-01-06      1.602174     26.548319      3.682552    128.642899  
2014-01-07      1.602174     27.718248      3.682552    127.703896  
2014-01-08      1.629798     26.908297      3.793029    130.520905  

[5 rows x 236 columns]